In [211]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [212]:
df=pd.read_csv('/kaggle/input/scrapped-lyrics-from-6-genres/lyrics-data.csv')

In [213]:
df.sample(5)

In [214]:
df.isnull().sum()

In [215]:
df.Idiom.value_counts()

In [216]:
def change(s):
    return s[1:len(s)-1]
df['Artist']=df['ALink'].apply(change)
df.head()

In [217]:
df.drop(['ALink','SLink','Idiom'],axis=1,inplace=True)

In [218]:
df.head()

In [219]:
ar_df=pd.read_csv('/kaggle/input/scrapped-lyrics-from-6-genres/artists-data.csv')
ar_df.head()

In [220]:
ar_df['Genre'].value_counts()

In [221]:
ar_df.drop(['Songs','Popularity','Link','Genres'],axis=1,inplace=True)

In [222]:
m_df=pd.merge(df,ar_df,on='Artist')

In [223]:
m_df['Genre'].value_counts()

In [224]:
m_df.sample(5)

In [225]:
len(df),len(ar_df)

In [226]:
len(m_df)

In [227]:
m_df.set_index('SName',inplace=True)

In [228]:
m_df.head()

In [229]:
ar_df.Genre.value_counts()

In [230]:
from nltk.tokenize import word_tokenize

In [231]:
l=[]
for ly in m_df['Lyric']:
    l.append(word_tokenize(ly))


In [232]:
for i in range(len(l)):
    l[i]=[j for j in l[i] if j.isalnum()]

In [233]:
for i in range(len(l)):
    for j in range(len(l[i])):
        l[i][j]=l[i][j].lower()

In [234]:
for i in range(len(l)):
    l[i]=' '.join(l[i])
l[:10]

In [235]:
m_df.head()

In [236]:
m_df['Lyric']=l

In [237]:
m_df.head()

In [238]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(m_df['Lyric'].values,m_df['Genre'].values,test_size=0.1)
vectorizer = CountVectorizer()
train_matrix=vectorizer.fit_transform(x_train)
test_matrix=vectorizer.transform(x_test)

In [239]:
test_matrix.shape

In [240]:
train_matrix.shape

y_train

In [241]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='liblinear')
model.fit(train_matrix,y_train)

In [242]:
y_pred=model.predict(test_matrix)